In [1]:
# interactive form to pick a ranking from dropdown and predict
# comments simple and lowercase no punctuation

import pandas as pd
import numpy as np
import json
from tensorflow import keras
import ipywidgets as widgets
from IPython.display import display, clear_output

# load meta model and csv
with open('../artifacts/metadata.json') as f:
    meta = json.load(f)
labels = meta['label_classes']
suit_classes = meta['suit_classes']
num_suits = meta['num_suits']
num_ranks = meta['num_ranks']
x_dim = meta['x_dim']

model = keras.models.load_model('../artifacts/best_so_far.keras')
df = pd.read_csv('../dataset/poker_hands.csv')

# helper to build one hot input same as prep
def row_to_x(row):
    scols = ['suit1','suit2','suit3','suit4','suit5']
    rcols = ['rank1','rank2','rank3','rank4','rank5']
    suit_to_idx = {s:i for i,s in enumerate(suit_classes)}
    suits = np.array([suit_to_idx[str(row[c])] for c in scols], dtype=np.int16)
    ranks = np.array([int(row[c]) for c in rcols], dtype=np.int16)
    key = ranks * 10 + suits
    order = np.argsort(key)
    ranks = ranks[order]
    suits = suits[order]
    r_oh = np.eye(num_ranks, dtype=np.uint8)[ranks - 1]   # 5 13
    s_oh = np.eye(num_suits, dtype=np.uint8)[suits]       # 5 k
    x = np.concatenate([s_oh.reshape(1, -1), r_oh.reshape(1, -1)], axis=1).astype('float32')
    return x, ranks, suits

# widgets
dd = widgets.Dropdown(options=labels, value=labels[0], description='ranking')
btn = widgets.Button(description='pick and predict')
out = widgets.Output()

def on_click(_):
    with out:
        clear_output()
        subset = df[df['ranking'] == dd.value]
        if subset.empty:
            print('no rows for this ranking')
            return
        row = subset.sample(1, random_state=np.random.randint(0, 1_000_000)).iloc[0]
        x, ranks_sorted, suits_sorted = row_to_x(row)
        if x.shape[1] != x_dim:
            print('feature dim mismatch expected', x_dim, 'got', x.shape[1])
            return
        pred = model.predict(x, verbose=0)
        pred_idx = int(np.argmax(pred, axis=1)[0])
        pred_label = labels[pred_idx]
        print('hand')
        for i in range(5):
            print(f'card{i+1} rank {int(ranks_sorted[i])} suit {suit_classes[int(suits_sorted[i])]}')
        print('true ranking', dd.value)
        print('predicted ranking', pred_label)

btn.on_click(on_click)
display(widgets.VBox([dd, btn, out]))


2025-10-26 03:23:43.831289: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-26 03:23:43.880015: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2025-10-26 03:23:44.695788: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-10-26 03:23:44.709335: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-10-26 03:23:44.711148: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-10-26 03:23:44.712815: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-10-26 03:23:44.720500: I tensorflow/core/platform/cpu_feature_g